In [3]:
from keys import FTX_API_KEY, FTX_API_SECRET
from datetime import datetime
import time
import hmac
from requests import Request, Session, Response

class FtxClient:


    def __init__(self, api_key=None, api_secret=None, subaccount_name=None):
            self._session = Session()
            self._api_key = api_key
            self._api_secret = api_secret
            self._subaccount_name = subaccount_name
            self._ENDPOINT = 'https://ftx.com/api/'


    def sign_request(self, request: Request):
        ts = int(time.time() * 1000)
        request = Request('GET', self._ENDPOINT)
        prepared = request.prepare()
        print(prepared)
        signature_payload = f'{ts}{prepared.method}{prepared.path_url}'.encode()
        signature = hmac.new(self._api_secret.encode(), signature_payload, 'sha256').hexdigest()

        request.headers['FTX-KEY'] = self._api_key
        request.headers['FTX-SIGN'] = signature
        request.headers['FTX-TS'] = str(ts)

    def send_request(self, method: str, path: str, **kwargs):
        request = Request(method, self._ENDPOINT + path, **kwargs)
        self.sign_request(request)
        response = self._session.send(request.prepare())
        return self._process_response(response)

    def _process_response(self, response: Response):
        try:
            data = response.json()
        except ValueError:
            response.raise_for_status()
            raise
        else:
            if not data['success']:
                raise Exception(data['error'])
            return data['result']



In [4]:
acc = FtxClient(api_key=FTX_API_KEY, api_secret=FTX_API_SECRET)
acc.send_request('Get', '/futures')

<PreparedRequest [GET]>


 100.0,
  'postOnly': False,
  'priceIncrement': 0.001,
  'sizeIncrement': 0.1,
  'type': 'perpetual',
  'underlying': 'UNI',
  'underlyingDescription': 'Uniswap Protocol Token',
  'upperBound': 33.078,
  'volume': 1431287.5,
  'volumeUsd24h': 44025441.9016},
 {'ask': 31.853,
  'bid': 31.765,
  'change1h': -0.004906710004062881,
  'change24h': 0.012239707518677475,
  'changeBod': 0.030554117037804248,
  'description': 'Uniswap Protocol Token March 2021 Futures',
  'enabled': True,
  'expired': False,
  'expiry': '2021-03-26T03:00:00+00:00',
  'expiryDescription': 'March 2021',
  'group': 'quarterly',
  'imfFactor': 0.001,
  'index': 31.435644720996326,
  'last': 31.84,
  'lowerBound': 29.864,
  'marginPrice': 31.84,
  'mark': 31.84,
  'moveStart': None,
  'name': 'UNI-0326',
  'perpetual': False,
  'positionLimitWeight': 200.0,
  'postOnly': False,
  'priceIncrement': 0.001,
  'sizeIncrement': 0.1,
  'type': 'future',
  'underlying': 'UNI',
  'underlyingDescription': 'Uniswap Protocol 